# EDBSCAM-ICR Parameter Tunning palco_2010

In [2]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import numpy as np
from BISGD import BISGD
from EDBSCAM_ICR import EDBSCAM_ICR
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt
import pandas as pd

## Functions


In [3]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [4]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_1.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]
Data = Data[:1000]


       Unnamed: 0  user_id  track_id            timestamp
0              87     4110     39059  2010-01-01 10:13:30
1              88     4110     36931  2010-01-01 10:14:19
2              89     4110     36360  2010-01-01 10:14:28
3              90     4110     38603  2010-01-01 10:14:31
4              91     4110     39062  2010-01-01 10:18:30
...           ...      ...       ...                  ...
32570      588644    66046     60491  2010-12-31 18:54:15
32571      588649    47201     51413  2010-12-31 18:56:51
32572      588650    66046     57049  2010-12-31 18:58:39
32573      588652    47201     74940  2010-12-31 19:01:51
32574      588658    47201     74798  2010-12-31 19:05:27

[32575 rows x 4 columns]


# Parameter Study

## Eps

In [15]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])


eps_array = [0.1,0.25,0.5,0.75,1.0]
#eps_array = [0.25,0.5,0.75,1.0]
num_clusters = 8
cl_num_iterations = 5
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
#eps = 0.4
MinPts = 5
k = 8
for ep in eps_array:
    print("eps:")
    print(ep)
    model = EDBSCAM_ICR(ImplicitData([],[]),  ep, MinPts, k, num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

eps:
0.1
start time 2022-09-16 11:20:23.718784


/home/mmelo/KControledIncDBSCAN/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/KControledIncDBSCAN/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.16666666666666666
end time 2022-09-16 11:30:20.399557
run time 0:09:56.680773

get tuple 6.949043273925781e-06
recommend 0.06062746047973633
eval_point 4.172325134277344e-05
update 0.059620688319206236
eps:
0.25
start time 2022-09-16 11:30:20.482527
npmean(resuls[Recall@20]) 0.16666666666666666
end time 2022-09-16 11:40:58.232045
run time 0:10:37.749518

get tuple 6.586933135986328e-06
recommend 0.054100354512532554
eval_point 0.00025133291880289715
update 0.06373115711212159
eps:
0.5
start time 2022-09-16 11:40:58.886143

Cluster1

Cluster1 Points:
-----------------
[['41572', '23345'], ['41572', '38875'], ['41572', '38871'], ['41572', '31808']]
4
-----------------

Cluster2

Cluster2 Points:
-----------------
[['1165', '23652'], ['1165', '23651'], ['1165', '33460']]
3
-----------------

Cluster3

Cluster3 Points:
-----------------
[['4298', '11662'], ['13914', '37970'], ['140', '23915'], ['140', '24334'], ['4298', '38503']]
5
-----------------

Cluster4

C

## Min

In [17]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])


min_array = [5,10,15,20,25,30]

num_clusters = 8
cl_num_iterations = 5
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
eps = 0.25
#MinPts = 5
k = 8
for min in min_array:
    print("min:")
    print(min)
    model = EDBSCAM_ICR(ImplicitData([],[]),  eps, min, k, num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

min:
5
start time 2022-09-16 14:57:44.488365
npmean(resuls[Recall@20]) 0.0
end time 2022-09-16 15:08:28.662763
run time 0:10:44.174398

get tuple 6.766963005065918e-06
recommend 0.04747457504272461
eval_point 4.3344497680664065e-05
update 0.0643824981212616
min:
10
start time 2022-09-16 15:08:29.311967
npmean(resuls[Recall@20]) 0.0
end time 2022-09-16 15:18:43.403590
run time 0:10:14.091623

get tuple 6.17527961730957e-06
recommend 0.062241315841674805
eval_point 3.814697265625e-05
update 0.06138630483150482
min:
15
start time 2022-09-16 15:18:44.052054
npmean(resuls[Recall@20]) 0.0
end time 2022-09-16 15:29:06.460308
run time 0:10:22.408254

get tuple 6.127381324768066e-06
recommend 0.0576249361038208
eval_point 4.029273986816406e-05
update 0.06221929078102112
min:
20
start time 2022-09-16 15:29:07.140974
npmean(resuls[Recall@20]) 0.0
end time 2022-09-16 15:39:50.786485
run time 0:10:43.645511

get tuple 6.954574584960937e-06
recommend 0.04204446928841727
eval_point 4.2983463832310266

## Iterations

In [19]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])


it_array = [5,10,15,10,25,30]

num_clusters = 8
#cl_num_iterations = 5
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
eps = 0.25
MinPts = 30 
k = 8
for it in it_array:
    print("it:")
    print(it)
    model = EDBSCAM_ICR(ImplicitData([],[]),  eps, min, k, num_clusters, it, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

it:
5
start time 2022-09-17 20:36:15.206052
npmean(resuls[Recall@20]) 0.14285714285714285
end time 2022-09-17 20:47:01.931313
run time 0:10:46.725261

get tuple 7.036352157592773e-06
recommend 0.05740863936288016
eval_point 4.3698719569614953e-05
update 0.06462040901184082
it:
10
start time 2022-09-17 20:47:02.659198
npmean(resuls[Recall@20]) 0.09090909090909091
end time 2022-09-17 20:58:04.781840
run time 0:11:02.122642

get tuple 7.305598258972168e-06
recommend 0.06377703493291681
eval_point 4.651329734108665e-05
update 0.06612976791858673
it:
15
start time 2022-09-17 20:58:05.576474
npmean(resuls[Recall@20]) 0.5
end time 2022-09-17 21:09:20.783796
run time 0:11:15.207322

get tuple 6.99918270111084e-06
recommend 0.05178523063659668
eval_point 3.6716461181640625e-05
update 0.06749841642379761
it:
10
start time 2022-09-17 21:09:21.610390
npmean(resuls[Recall@20]) 0.0
end time 2022-09-17 21:20:30.183202
run time 0:11:08.572812

get tuple 7.143282890319824e-06
recommend 0.04187984466552

## Learning Rate

In [20]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])


lr_array = [0.0001,0.001,0.01,0.1]

num_clusters = 8
cl_num_iterations = 15
#cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
eps = 0.25
MinPts = 30 
k = 8
for lr in lr_array:
    print("lr:")
    print(lr)
    model = EDBSCAM_ICR(ImplicitData([],[]),  eps, min, k, num_clusters, cl_num_iterations, lr, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lr:
0.0001
start time 2022-09-17 22:07:28.837341
npmean(resuls[Recall@20]) 0.16666666666666666
end time 2022-09-17 22:19:45.497870
run time 0:12:16.660529

get tuple 7.423114776611328e-06
recommend 0.057602326075236
eval_point 4.478295644124349e-05
update 0.07361913242340087
lr:
0.001
start time 2022-09-17 22:19:46.646598
npmean(resuls[Recall@20]) 0.25
end time 2022-09-17 22:30:40.245504
run time 0:10:53.598906

get tuple 6.933760643005371e-06
recommend 0.023956894874572754
eval_point 4.374980926513672e-05
update 0.06533855559825898
lr:
0.01
start time 2022-09-17 22:30:40.627995

Cluster1

Cluster1 Points:
-----------------
[['4298', '4347'], ['41572', '38875'], ['41572', '38871'], ['41572', '31808'], ['41572', '23345'], ['41572', '23343'], ['41572', '15506'], ['41572', '23341'], ['4298', '38503'], ['4298', '11662'], ['4298', '11667'], ['4298', '11663'], ['4298', '30324'], ['4298', '4348'], ['4298', '4344'], ['4298', '4346'], ['4298', '4345']]
17
-----------------
npmean(resuls[Recall@

## Regularization

In [22]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])


reg_array = [0,0.0001,0.001,0.01,0.1]

num_clusters = 8
cl_num_iterations = 15
cl_learn_rate =0.001
#cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
eps = 0.25
MinPts = 30 
k = 8
for reg in reg_array:
    print("lr:")
    print(lr)
    model = EDBSCAM_ICR(ImplicitData([],[]),  eps, min, k, num_clusters, cl_num_iterations, cl_learn_rate, reg, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lr:
0.1
start time 2022-09-17 23:47:48.457764
npmean(resuls[Recall@20]) 0.0
end time 2022-09-17 23:58:26.467158
run time 0:10:38.009394

get tuple 6.55059814453125e-06
recommend 0.04525035619735718
eval_point 5.525350570678711e-05
update 0.06377180695533752
lr:
0.1
start time 2022-09-17 23:58:26.714352
npmean(resuls[Recall@20]) 0.25
end time 2022-09-18 00:08:38.581100
run time 0:10:11.866748

get tuple 5.961394309997559e-06
recommend 0.059625834226608276
eval_point 3.680586814880371e-05
update 0.061128655219078065
lr:
0.1
start time 2022-09-18 00:08:38.821424
npmean(resuls[Recall@20]) 0.0
end time 2022-09-18 00:18:28.269974
run time 0:09:49.448550

get tuple 5.653810501098633e-06
recommend 0.04989186922709147
eval_point 3.425280253092448e-05
update 0.058920312809944156
lr:
0.1
start time 2022-09-18 00:18:28.504586
npmean(resuls[Recall@20]) 0.16666666666666666
end time 2022-09-18 00:28:26.141028
run time 0:09:57.636442

get tuple 5.78911304473877e-06
recommend 0.057047486305236816
eval_